<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --quiet spacy
!pip install "spacy-pkuseg>=0.0.27,<0.1.0"
!python -m spacy download zh_core_web_lg
# !wget https://github.com/explosion/spacy-models/releases/download/zh_core_web_md-3.1.0/zh_core_web_md-3.1.0.tar.gz
# !tar xvfz zh_core_web_md-3.1.0.tar.gz
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 6.4 MB 13.2 MB/s 
     |████████████████████████████████| 10.1 MB 47.0 MB/s 
     |████████████████████████████████| 456 kB 46.5 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 621 kB 54.6 MB/s 
     |████████████████████████████████| 2.4 MB 11.2 MB/s 
2021-08-10 10:16:52.668046: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 603.8 MB 8.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [2]:
import spacy

# spacyModel='zh_core_web_md-3.1.0/zh_core_web_md/zh_core_web_md-3.1.0'
nlp = spacy.load('zh_core_web_lg')

doc = nlp('恆指漲3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。')

for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

3.3% 3 7 PERCENT
香港 9 11 GPE
9% 29 31 PERCENT
去年7月 34 38 DATE


In [3]:
import random
from spacy.training.example import Example

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN'),(12,16,'PERCENT')          
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),
       (17,19,'STOCK'),(24,28,'PERCENT')       
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(3,8,'STOCK'),(11,14,'DATE'),(15,17,'DATE'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '美股異動受港股收跌拖累和黃低開低走跌14%。',
    {
        'entities':
        [
         (0,2,'FIN'),(5,7,'FIN'),(7,11,'FIN'),(11,13,'STOCK'),(13,17,'FIN'),(18,21,'PERCENT')
        ]
    }
  ),
  (
    '港隊於東京奧運創歷史佳績。',
   {
       'entities': [(3,7,'EVENT')]
   }
  )
]

ner = nlp.get_pipe('ner')

ner.add_label('STOCK')
ner.add_label('FIN')

epoch = 50

# get names of other pipes to disable them during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print(f'Pipes: {other_pipes}')
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA, size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.1,losses=losses)
    print("Losses", losses)

nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

Pipes: ['tok2vec', 'tagger', 'parser', 'attribute_ruler']


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "美股異動受港股收跌拖累和黃低開低走跌14%。" with entities "[(0, 2, 'FIN'), (5, 7, 'FIN'), (7, 11, 'FIN'), (11...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 52.65751765809154}
Losses {'ner': 46.38638983658211}
Losses {'ner': 42.57746655248047}
Losses {'ner': 35.615400750830304}
Losses {'ner': 26.869933511246927}
Losses {'ner': 23.37840343742573}
Losses {'ner': 20.659830483968108}
Losses {'ner': 16.541440239818712}
Losses {'ner': 13.509798728961954}
Losses {'ner': 35.23600546289056}
Losses {'ner': 37.20949145126947}
Losses {'ner': 17.71673724299232}
Losses {'ner': 18.074394019072876}
Losses {'ner': 12.461618219913547}
Losses {'ner': 9.463106532292866}
Losses {'ner': 7.210770292236166}
Losses {'ner': 4.0609108116201895}
Losses {'ner': 3.2732902203076586}
Losses {'ner': 0.785588150891317}
Losses {'ner': 0.19366680900711586}
Losses {'ner': 0.009122319347227651}
Losses {'ner': 0.0024671828123830775}
Losses {'ner': 0.00042520705823494766}
Losses {'ner': 0.0006473395573262589}
Losses {'ner': 0.00030965911216264885}
Losses {'ner': 1.3655284462822303e-05}
Losses {'ner': 2.1910663977341574e-06}
Losses {'ner': 7.152378273657893e-05}
Lo

In [4]:
# for text, annot in TRAIN_DATA:
#     print(text,annot)
#     doc = nlp.make_doc(text)
#     biluo = spacy.training.offsets_to_biluo_tags(doc, annot["entities"])
#     print([t.text for t in doc], biluo)

In [5]:
# Let's give our trained model a try!

texts = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

nlp_updated = spacy.load('saved_model')

from spacy import displacy
for text in texts:
  doc = nlp_updated(text)
  displacy.render(doc,jupyter=True,style="ent")

